In [2]:
# %%
import time
import numpy as np
import pickle
from numpy.linalg import det

from modules import CMINE_lib 
# from Guassian_variables import Data_guassian

import pandas as pd
from scipy.stats import multivariate_normal
import itertools

np.random.seed(37)
from scipy import stats
from sklearn.neighbors import KernelDensity

import math

import torch 
import torch.nn as nn
import torch.nn.functional as F

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [3]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        if isinstance(sum_exp, Number):
            return m + math.log(sum_exp)
        else:
            return m + torch.log(sum_exp)

In [4]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [5]:
from modules.CMI import DR_CMI, CDL_CMI
from modules import CMINE

In [6]:
a

NameError: name 'a' is not defined

In [20]:
Dim = 5
batch_size = 60
#dataset = CMINE.create_dataset_DGP( Dim=5, N=batch_size)
dataset = CMINE_lib.create_dataset_DGP_binary_A( Dim=5, N=batch_size * 10)
s_t = torch.from_numpy(dataset[0]).float().cuda()
s_next = torch.from_numpy(dataset[1]).float().cuda()
a = torch.from_numpy(dataset[2]).float().cuda()

CMI_LDR, CMI_DV, CMI_NWJ = CMINE.estimate_CMI_dim(dim = Dim, n =  batch_size * 10, b_size= batch_size, epoch=10, s =1, t=1, arrng = [[0],[1],[2]]  , dataset=dataset)

Duration of data preparation:  4.76837158203125e-07  seconds
Print
LDR= [0.037544811853966235, 0.06457196457757552, 0.03476451698449177, 0.009375336523248748, 0.0627677772592072, 0.06116905937344834, 0.05828500824830405, -0.04386110140835794, 0.01098467926155379, -0.027625196262036855]
DV= [0.015759347263096895, -0.03128905878574251, 0.02497415825909513, -0.00994539193714241, 0.017595272600274, -0.008790621807393317, -0.0031406962371052505, -0.0062450552815601496, 0.00832774747706886, -0.04614101010164662]
NWJ= [0.01552031134913201, -0.036034130253708385, 0.024926075910733703, -0.010133245080733433, 0.016559457106613218, -0.011295880489089027, -0.0050664830994262555, -0.00694375063292596, 0.008324214705730526, -0.04631349067753732]
True=Todo
Duration:  0.014765739440917969  seconds


In [14]:
len(CMI_LDR)

20

In [7]:
sample_dim = 2*Dim

hidden_size = 15
learning_rate = 0.005
training_steps = 10

cubic = False 


In [8]:
def train_dr(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_dr = DR_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_dr = torch.optim.Adam(model_dr.parameters(), learning_rate)
    dr_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_dr.eval()
        drs = model_dr(batch_x, batch_y)
        #mi_est_values.append(cmi)
        dr_est_values.append(drs)
        model_dr.train() 

        model_loss = model_dr.learning_loss(batch_x, batch_y)

        optimizer_dr.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_dr.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return dr_est_values

In [9]:
def train_cdl(N = 64, training_steps = 10):
    torch.cuda.empty_cache()
    model_cdl = CDL_CMI(sample_dim + 1, sample_dim, hidden_size).cuda()
    optimizer_cdl = torch.optim.Adam(model_cdl.parameters(), learning_rate)
    cdl_est_values = []
    for step in range(training_steps):
        #batch_x, batch_y = sample_correlated_gaussian(rho, dim=sample_dim, batch_size = batch_size, to_cuda = True, cubic = cubic)
        dataset = CMINE.create_dataset_DGP_binary_A(Dim=Dim, N=N)
        s_t = torch.from_numpy(dataset[0]).float().cuda()
        s_next = torch.from_numpy(dataset[1]).float().cuda()
        a = torch.from_numpy(dataset[2]).float().cuda()

        batch_x = torch.cat([s_t,a], dim=1)
        batch_y = s_next
        model_cdl.eval()
        cdl_cmi = model_cdl(batch_x, batch_y)
        cdl_est_values.append(cdl_cmi)
        model_cdl.train() 

        model_loss = model_cdl.learning_loss(batch_x, batch_y)

        optimizer_cdl.zero_grad()
        model_loss.backward(retain_graph=True)
        optimizer_cdl.step()

        del batch_x, batch_y
        torch.cuda.empty_cache()
    return cdl_est_values

In [10]:
cdl_est_values = train_cdl()


In [11]:
dr_est_values = train_dr()

In [23]:
N = 64
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[ 0.38522759  0.63885367  0.10865278  0.61746345  0.16564892 43.69571261
  0.3061948   0.16505242  7.28289756  1.48086643]
[1.08386106e+03 2.45151242e+02 1.71678050e+02 1.88725865e+02
 1.87000385e+02 2.36553918e+08 5.12960772e+05 4.43446152e+02
 3.29352425e+06 1.26835742e+03]


In [24]:
N = 128
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[9.57500424e+02 8.70394940e+00 8.78661077e+00 8.64446817e+00
 8.73174875e+00 3.84186669e+04 1.60076906e+04 2.95070732e+02
 3.33992870e+04 5.66890174e+09]
[8.92431697e-01 5.49440482e-01 2.40295252e+01 7.05529320e-01
 5.19527063e-01 3.83233965e+05 2.35686785e+01 3.58856802e+04
 3.72005749e+01 1.11821590e+02]


In [22]:
N = 32
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[6.44155857e+05 6.42998652e+05 6.43009391e+05 6.43004548e+05
 6.43049284e+05 8.56980191e+07 1.02583050e+15 1.93417102e+08
 7.82352461e+07 3.49760795e+08]
[1.26137582e+00 2.85512393e+01 4.84103487e+01 1.04888001e+00
 1.11722267e+00 2.31955002e+03 8.30490970e+00 1.05053730e+06
 1.63817763e+03 1.73307146e+03]


In [20]:
N = 16
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[7.13807219e-01 6.97927269e+00 3.75521322e+01 3.87761343e+01
 1.29613435e+01 1.55601522e+01 1.47023124e+00 8.46078698e+01
 3.38975972e+03 4.86156490e+00]
[2.94362882e+07 5.55052156e+06 5.54733155e+06 5.54933957e+06
 5.55410870e+06 3.25353404e+09 1.37534833e+10 7.43379986e+11
 1.77425236e+10 3.11015693e+16]


In [21]:
N = 8
cdl_est_values = train_cdl(N)
dr_est_values = train_dr(N)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.43795723e+04 3.03923606e+01 1.32917732e+00 5.11034797e+02
 4.44193287e+04 1.97822264e+02 2.01944086e+05 5.38163164e+00
 5.57309857e+02 6.09261700e+01]
[ 0.50340626  0.40486414  0.41866397  0.46100656  0.41739924 35.34570122
 89.93095651 11.26144007  6.79382749 72.36125969]


# Training Step

In [15]:
N = 64
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[7.90087952e+02 7.85717166e+02 8.86630148e+02 7.85646969e+02
 1.60624307e+03 6.54722027e+07 4.60514869e+05 8.44506005e+04
 2.02980041e+05 5.45358667e+05]
[1.26068347e+07 6.77329778e+11 1.26400620e+07 3.45984030e+07
 1.27147874e+07 1.91223624e+15 2.71988959e+09 3.92338222e+12
 4.32217656e+14 1.64770941e+11]


In [14]:
N = 32
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[4.18966604e+00 3.15982827e+06 8.61560204e+02 2.23745444e+01
 2.57475600e+04 2.69649200e+05 7.87265592e+02 6.23005884e+03
 7.46181203e+04 1.20451452e+03]
[2.80374949e+10 3.08702256e+10 2.81252140e+10 2.81249405e+10
 2.81251843e+10 2.43591543e+13 2.16347626e+13 2.92307035e+19
 3.14428485e+12 7.73048805e+13]


In [19]:
print(np.array(cdl_est_values[-50:]).mean(axis=0))
print(np.array(dr_est_values[-50:]).mean(axis=0))

[1.57217472e+03 1.57109104e+03 1.77133819e+03 1.57102237e+03
 1.81083862e+03 1.30944404e+08 9.21023368e+05 1.68589411e+05
 4.05958852e+05 1.06707031e+06]
[2.51348668e+07 1.35465464e+12 2.52112564e+07 2.52614051e+07
 2.53483606e+07 3.82447132e+15 5.32261770e+09 7.53837592e+12
 8.64435300e+14 3.29537708e+11]


In [31]:
N = 526
training_step = 100
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[1.50006065e+01 2.18871596e+02 1.10105989e+07 5.11114454e+02
 1.06051950e+02 9.00031509e+01 1.04145438e+04 3.52512225e+03
 5.52125692e+03 2.41971664e+04]
[7.68950225e+06 7.74432824e+06 7.91731634e+06 7.70377130e+06
 7.70934032e+06 7.88427070e+08 4.68894041e+16 1.52884448e+10
 1.34091251e+11 2.40433729e+10]


In [30]:
N = 526
training_step = 10
cdl_est_values = train_cdl(N, training_step)
dr_est_values = train_dr(N, training_step)
print(np.array(cdl_est_values).mean(axis=0))
print(np.array(dr_est_values).mean(axis=0))

[2.41623130e-01 5.83663910e-01 2.62432172e-01 1.86141613e-01
 2.88988191e+02 2.00060656e+01 5.57397880e+00 2.69901884e+01
 2.13138771e-01 1.53580540e+00]
[1.34774063e+06 7.49846064e+03 7.87508115e+03 7.57209025e+03
 7.50388969e+03 5.57463735e+05 1.85607135e+04 3.51530271e+09
 1.84042864e+05 2.73516473e+06]
